In [98]:
import numpy as np
import csv
from scipy.interpolate import CubicSpline, PPoly
import matplotlib.pyplot as plt

debug = True
def dbp(*args):
    # Debug mode printouts
    if debug:
        print(" ".join([str(a) for a in args]))

In [99]:
class Fluid:
    def __init__(self, h_file, cp_file, s_file=None, r_file=None):
        self.Pc = None
        self.Tc = None
        self.s0 = None
        self.T0 = 400
        self.P0 = 1
        self.gamma = None
        self.R = 1
        
        with open(cp_file, mode='r', newline='') as f:
            reader = csv.reader(f)
            cp_data = np.array([[s.replace(',', '') for s in r] for r in reader])
        cptab = Table(P=cp_data[0,1:].astype(float), T=cp_data[1:,0].astype(float), data=cp_data[1:,1:].astype(float))
        self.tables = dict()
        self.tables['h'] = self.read_table(h_file)
        self.tables['cp'] = self.read_table(cp_file)
        self.tables['r'] = self.read_table(r_file)
        self.tables['s'] = self.read_table(s_file)
    
    # reads in csv file
    def read_table(self, fname):
        with open(fname, mode='r', newline='') as f:
            reader = csv.reader(f)
            data = np.array([[s.replace(',', '') for s in r] for r in reader])
        table = Table(P=data[0,1:].astype(float), T=data[1:,0].astype(float), data=data[1:,1:].astype(float))
        return table
    
    # Setting up API to return a full state with all properties based upon only two values
    def state(self, **kwargs):
    # e.g.:
        # pump_exit = H2.state(P=5, T=800); print(pump_exit.h)
        # plt.plot(range(1, 5), [H2.state(pressure=Pi, s=1800).v for Pi in range(1, 5)])
        if len(kwargs)==2:
            kw1 = list(kwargs.keys())[0]
            kw2 = list(kwargs.keys())[1]
        else:
            raise ValueError("Two properties are required to determine the fluid state")
        check = {'p':('P', 'PRESSURE', 'PRESS'),
                 't':('T', 'TEMPERATURE', 'TEMP'),
                 's':('S', 'ENTROPY'), 
                 'h':('H', 'ENTHALPY'),
                 'v':('V', 'VOLUME', 'SPECIFIC_VOLUME', 'VOL'), 
                 'r':('R', 'RHO', 'DENSITY')}
        input_props = dict()
        for key, vals in check.items():
            if kw1.upper() in vals:
                input_props[key] = int(kwargs[kw1])
            elif kw2.upper() in vals:
                input_props[key] = int(kwargs[kw2])
        dbp("Input state properties:", input_props)
        if len(input_props)!=2:
            raise ValueError("Input properties not recognized")
        return FluidState(self, input_props)

In [100]:
class Process:
    def __init__(self, state1, state2):
        pass

In [101]:
class FluidState:
    def __init__(self, fluid, props):
        self.P = None
        self.T = None
        self.h = None
        self.v = None
        self.r = None
        self.s = None
        self.cp = None
        pt_count = 0
        keys = list(props.keys())
        if 'v' in keys: # convert specific volume to density
            props['r'] = 1 / props['v']
            keys.remove('v')
            keys.append('r')
        
        if 't' in keys:
            self.T = props['t']
            pt_count += 1
        if 'p' in keys:
            self.P = props['p']
            pt_count += 1
        
        # Solve for both pressure and temperature if needed
        if pt_count == 0:
            self.P, self.T = self.double_solve(props[keys[0]], fluid.tables[keys[0]], props[keys[1]], fluid.tables[keys[1]])
        
        # Solve for only one of either pressure or temperature if needed
        if pt_count == 1:
            if self.T is None:
                other_key = keys[keys!='p'][0]
                self.T = fluid.tables[other_key].get_t( P1=self.P, Y1=props[other_key])
                
            elif self.P is None:
                other_key = keys[keys!='t'][0]
                self.P = fluid.tables[other_key].get_t( P1=self.P, Y1=props[other_key])
        dbp("Pressure:", self.P, "--- Temperature:", self.T)
        
        # Set specific heat property
        self.cp = fluid.tables['cp'].interp(P1=self.P, T1=self.T)
        
        # Set density property
        if 'r' in keys:
            self.r = props['r']
        else:
            self.r = fluid.tables['r'].interp(P1=self.P, T1=self.T)
        self.v = 1 / self.r
        
        # Set enthalpy property
        if 'h' in keys:
            self.h = props['h']
        else:
            self.h = fluid.tables['h'].interp(P1=self.P, T1=self.T)
        
        # set entropy property
        if 's' in keys:
            self.s = props['s']
        else:
            self.s = fluid.tables['s'].interp(P1=self.P, T1=self.T)
    
    
    # solve for both pressure and temperature as a function of other variables
    def double_solve(self, v1, tab1, v2, tab2):
        intcpt1 = dict()
        cs1 = tab1.cs_t
        for i in range(len(cs1)):
            T = tab1.combine(cs1[i].solve(v1))
            if len(T)==0:
                pass
            elif len(T)==1:
                intcpt1[tab1.P[i]] = T[0]
            else:
                raise ValueError("No Unique Solution Found")
        dbp(list(intcpt1.keys()), list(intcpt1.values()))
        func1 = CubicSpline(list(intcpt1.keys()), list(intcpt1.values()), extrapolate=False)
        
        intcpt2 = dict()
        cs2 = tab2.cs_t
        for i in range(len(cs2)):
            T = tab2.combine(cs2[i].solve(v2))
            if len(T)==0:
                pass
            elif len(T)==1:
                intcpt2[tab2.P[i]] = T[0]
            else:
                raise ValueError("No Unique Solution Found")
        dbp(list(intcpt2.keys()), list(intcpt2.values()))
        func2 = CubicSpline(list(intcpt2.keys()), list(intcpt2.values()), extrapolate=False)
        
        c_new = func1.c - func2.c
        difference = PPoly(c=c_new, x=func1.x)
        pressure_values = tab1.combine(difference.solve(0))
        if len(pressure_values)!=1:
            raise ValueError("No Unique Solution Found")
        else:
            temperature = func1(pressure_values[0])
            return pressure_values[0], temperature
                
    # control printout behavior of fluid state
    def __str__(self):
        pstr = [f"Fluid state with properties:",
                f"[P] Pressure: {self.P}",
                f"[T] Temperature: {self.T}",
                f"[h] Enthalpy: {self.h}",
                f"[r] Density: {self.r}",
                f"[v] Specific Volume: {self.v}",
                f"[s] Entropy: {self.s}"]
        return "\n\t".join(pstr)


In [102]:
# property table which contains some data indexed to pressure (columns) and temperature (rows)
class Table:
    def __init__(self, P, T, data):
        self.P = P
        self.T = T
        self.data = data
        self.tol = 1e-5 # tolerance for removing duplicate values
        
        # create spline fits as a function of pressure at specified temperatures
        self.cs_p = [] 
        for i in range(len(self.T)):
            self.cs_p.append(CubicSpline(self.P, self.data[i,:], extrapolate=False))
            
        # create spline fits as a function of temperature at specified pressures    
        self.cs_t = []                   
        for j in range(len(self.P)): 
            self.cs_t.append(CubicSpline(self.T, self.data[:,j], extrapolate=False))
    
    # get property value from pressure and temperature
    def interp(self, P1, T1):
        FP = self.func_p(T1)
        FT = self.func_t(P1)
        return (FP(P1) + FT(T1)) / 2
    
    # Property as a function of temperature at specified pressure P1
    def func_t(self, P1):
        at_p1 = []
        for i in range(len(self.T)):
            at_p1.append(self.cs_p[i](P1))
        return CubicSpline(self.T, at_p1, extrapolate=False)
    
    # Property as a function of pressure at specified temperature T1
    def func_p(self, T1):
        at_t1 = []
        for j in range(len(self.P)):
            at_t1.append(self.cs_t[j](T1))
        return CubicSpline(self.P, at_t1, extrapolate=False)
    
    # get pressure from temperature and property value (not recommended for enthalpy tables)
    def get_p(self, T1, Y1):
        func = self.func_p(T1)
        solns = self.combine(func.solve(Y1))
        if len(solns)!=1:
            print(solns)
            raise ValueError("No Unique Solution Found")
        return solns[0]
    
    # get temperature from pressure and property value
    def get_t(self, P1, Y1):
        func = self.func_t(P1)
        solns = self.combine(func.solve(Y1))
        if len(solns)!=1:
            print(solns)
            raise ValueError("No Unique Solution Found")
        return solns[0]
    
    # remove duplicate and NaN solutions
    def combine(self, vals):
        unique = []
        for i in range(len(vals)):
            for j in range(i,len(vals)):
                if i!=j and (abs(vals[i] - vals[j]) < self.tol):
                    break
                if np.isnan(vals[i]):
                    break
            else:
                unique.append(vals[i])
        return unique

In [103]:
H2 = Fluid("H2_data_h.csv", "H2_data_Cp.csv", "H2_data_h.csv", "H2_data_rho_placeholder.csv")  
print(H2.tables['r'].data)
a = H2.state(h=-100, rho=.0001)
print(a)

Input state properties: {'h': -100, 'r': 0}
[0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 30.0, 100.0, 300.0] [2763.2505645879255, 2911.9777382825673, 3093.038147482486, 3277.7424020778153, 3503.93923423576, 3736.7394903090503, 4025.365252592932, 4325.138607297019, 4701.356711273467, 5094.118238100766, 5589.080319653579, 6106.014817443672]
[] []


ValueError: `x` must contain at least 2 elements.